# MLOps: Deploy and Monitor

In [12]:
import os
import wml_sdk_utils as wml_util
import wos_sdk_utils as wos_util
import ws_utils as ws_util

wos_client = wos_util.get_client()

/opt/conda/envs/Python-3.8-CUDA/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/envs/Python-3.8-CUDA/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## User Configuration
### deployment admin to decide

In [37]:
# decide which wml space & openscale deployment service provider to use
WML_SPACE_ID = '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7' 
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"

# decide as you wish
MODEL_NAME = 'DeepLIIF wendy' # just give it name :)
DEPLOYMENT_NAME = 'deepliif-wendy' # no space, and be careful about special characters (some may not work)

In [39]:
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)

Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'


{'deployment_id': None,
 'deployment_space_id': '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7',
 'model_asset': 'Test_Model_wendy_ws_serialized2.zip',
 'model_name': 'DeepLIIF rich test',
 'openscale_custom_metric_provider': {'generic_metrics': {'dir_gt': 'DeepLIIF_Datasets/model_eval/gt_images',
   'dir_pred': 'DeepLIIF_Datasets/model_eval/model_images',
   'most_recent': 1,
   'thresholds': {'num_images_recent_ground_truth': {'threshold': [5.0,
      'lower']},
    'num_images_recent_predicted': {'threshold': [40.0, 'lower']},
    'num_images_total_ground_truth': {'threshold': [5.0, 'lower']},
    'num_images_total_predicted': {'threshold': [40.0, 'lower']}},
   'volume_display_name': 'AdditionalDeepLIIFVolume'},
  'segmentation_metrics': {'dir_gt': 'DeepLIIF_Datasets/model_eval/gt_images',
   'dir_pred': 'DeepLIIF_Datasets/model_eval/model_images',
   'most_recent': 5,
   'thresholds': {'Dice': {'threshold': [20.0, 'lower']},
    'Dice_negative': {'threshold': [0.2, 'lower']},
    'Dice_pos

### input from model owner: deployment related

In [3]:
# model and script files for deployment
PATH_MODEL = '/mnts/AdditionalDeepLIIFVolume/deepliif-ws-output/Test_Model_wendy_ws_serialized2'
PATH_DEPENDENCY = '/userfs/wmla-tutorial/deployment-wmla/edi_deployment_files/deepliif-base'

# WMLA deployment information
RESOURCE_CONFIGS = {'enable_gpus':'True',
                    'n_cpus':8,
                    'memory_allocation':10000,
                    'n_replicas':1,
                    'n_min_kernels':1,
                    'task_execution_timeout':2*60}

### input from model owner: monitor related

In [5]:
# openscale monitor related
DIR_GT = 'DeepLIIF_Datasets/model_eval/gt_images'
DIR_PRED = 'DeepLIIF_Datasets/model_eval/model_images'
VOLUME_DISPLAY_NAME = 'AdditionalDeepLIIFVolume'

# use the function to get default thresholds if default thresholds are desired
THRESHOLDS_GENERIC = wos_util.get_default_thresholds('generic_metrics',wos_client)

THRESHOLDS_SEGMENTATION = {'precision': {'threshold': [60.0, 'lower']},
 'precision_positive': {'threshold': [0.6, 'lower']},
 'precision_negative': {'threshold': [0.6, 'lower']},
 'recall': {'threshold': [15.0, 'lower']},
 'recall_positive': {'threshold': [0.15, 'lower']},
 'recall_negative': {'threshold': [0.15, 'lower']},
 'f1': {'threshold': [45.0, 'lower']},
 'f1_positive': {'threshold': [0.45, 'lower']},
 'f1_negative': {'threshold': [0.45, 'lower']},
 'Dice': {'threshold': [20.0, 'lower']},
 'Dice_positive': {'threshold': [0.20, 'lower']},
 'Dice_negative': {'threshold': [0.20, 'lower']},
 'IOU': {'threshold': [12.0, 'lower']},
 'IOU_positive': {'threshold': [0.12, 'lower']},
 'IOU_negative': {'threshold': [0.12, 'lower']},
 'PixAcc': {'threshold': [35.0, 'lower']},
 'PixAcc_positive': {'threshold': [0.35, 'lower']},
 'PixAcc_negative': {'threshold': [0.35, 'lower']}}

/opt/conda/envs/Python-3.8-CUDA/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'internal-nginx-svc'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


### deploy admin to modify `'openscale_custom_metric_provider'` section 
Make sure you added all the monitors you need / are asked to use.

In [6]:
# use double brackets to indicate that it's not for format to fill in a value but an actual bracket
metadata = """
{{{model_asset_id}:{{
                 'model_asset': {model_asset_name},
                 'model_name': {MODEL_NAME},
                 'deployment_id': None,
                 'deployment_space_id':{WML_SPACE_ID},
                 'openscale_subscription_id': None,
                 'openscale_custom_metric_provider': 
                     {{'segmentation_metrics':{{'dir_gt': {DIR_GT},
                                              'dir_pred': {DIR_PRED},
                                              'volume_display_name': {VOLUME_DISPLAY_NAME},
                                              'most_recent': 5,
                                              'thresholds': {THRESHOLDS_SEGMENTATION}}},
                      'generic_metrics': {{'dir_gt': {DIR_GT},
                                          'dir_pred': {DIR_PRED},
                                          'volume_display_name': {VOLUME_DISPLAY_NAME},
                                          'most_recent': 1,
                                          'thresholds': {THRESHOLDS_GENERIC}}}
                     }},
                'wmla_deployment':{{'deployment_name':{DEPLOYMENT_NAME},
                                   'deployment_url':{deployment_url},
                                   'dependency_filename':{dependency_asset_name},
                                   'volume_display_name':{VOLUME_DISPLAY_NAME},
                                   'resource_configs':{RESOURCE_CONFIGS}}}
             }}}}
"""

### settings you don't need to worry about

In [ ]:
data_mart_id = '00000000-0000-0000-0000-000000000000'
deployment_url = f'https://wmla-inference-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/inference/{DEPLOYMENT_NAME}'
subscription_name = f"{DEPLOYMENT_NAME} Monitor" # a new subscription name

model_asset_name = os.path.basename(PATH_MODEL)
if os.path.isdir(PATH_MODEL):
    model_asset_name += '.zip'
        
dependency_asset_name = os.path.basename(PATH_DEPENDENCY)
if os.path.isdir(PATH_DEPENDENCY):
    dependency_asset_name += '.zip'

## 1. Publish Assets to WML Deployment Space

In [8]:
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)
wml_client.version

'1.0.141'

In [8]:
# wml_util.list_files(wml_client)

In [9]:
wml_util.upload_batch([PATH_MODEL,PATH_DEPENDENCY],wml_client,overwrite=False)

Creating data asset...
SUCCESS
Finished publishing /userfs/tmp/Test_Model_wendy_ws_serialized2.zip as Test_Model_wendy_ws_serialized2.zip
Finished deleting existing old assets with the same name
Creating data asset...
SUCCESS
Finished publishing /userfs/tmp/deepliif-base.zip as deepliif-base.zip
Finished deleting existing old assets with the same name


## 2. Update Metadata yaml for Main Model Asset
This yaml file contains deployment information about a corresponding model file, plus needed input for a custom metric provider to use.

In [10]:
def metadata_yml_fill(metadata_str):
    variables = list(globals())
    d_variables = {k:repr(eval(k)) for k in variables}
    metadata_str_filled = metadata_str.format(**d_variables)
    
    return eval(metadata_str_filled)

In [11]:
data_assets = wml_util.list_files(wml_client,keep_only_latest=True)
model_asset_id = [k for k,v in data_assets.items() if v == model_asset_name][0]

In [12]:
metadata = metadata_yml_fill(metadata)
# metadata

In [14]:
metadata

{'d289073f-2079-4bce-ba93-cc4fa33eb0d7': {'model_asset': 'Test_Model_wendy_ws_serialized2.zip',
  'model_name': 'DeepLIIF wendy',
  'deployment_id': None,
  'deployment_space_id': '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7',
  'openscale_subscription_id': None,
  'openscale_custom_metric_provider': {'segmentation_metrics': {'dir_gt': 'DeepLIIF_Datasets/model_eval/gt_images',
    'dir_pred': 'DeepLIIF_Datasets/model_eval/model_images',
    'volume_display_name': 'AdditionalDeepLIIFVolume',
    'most_recent': 5,
    'thresholds': {'precision': {'threshold': [60.0, 'lower']},
     'precision_positive': {'threshold': [0.6, 'lower']},
     'precision_negative': {'threshold': [0.6, 'lower']},
     'recall': {'threshold': [15.0, 'lower']},
     'recall_positive': {'threshold': [0.15, 'lower']},
     'recall_negative': {'threshold': [0.15, 'lower']},
     'f1': {'threshold': [45.0, 'lower']},
     'f1_positive': {'threshold': [0.45, 'lower']},
     'f1_negative': {'threshold': [0.45, 'lower']},
   

In [13]:
wml_util.metadata_yml_add(metadata,wml_client,overwrite=True)

Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'
Key d1a14d91-d223-4895-bc4d-4badfbb80172 already exists in yaml file deployment_metadata.yml, updating the values...
Writing new metadata in..
Creating data asset...
SUCCESS
Finished publishing tmp/deployment_metadata.yml as deployment_metadata.yml
Finished deleting existing old assets with the same name


In [14]:
wml_util.metadata_yml_load(wml_client)[model_asset_id]

Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'


{'deployment_id': None,
 'deployment_space_id': '81c825b4-a1ae-4c1e-8cf3-51b6e3b301b7',
 'model_asset': 'Test_Model_wendy_ws_serialized.zip',
 'model_name': 'DeepLIIF rich test',
 'openscale_custom_metric_provider': {'generic_metrics': {'dir_gt': 'DeepLIIF_Datasets/model_eval/gt_images',
   'dir_pred': 'DeepLIIF_Datasets/model_eval/model_images',
   'most_recent': 1,
   'thresholds': {'num_images_recent_ground_truth': {'threshold': [5.0,
      'lower']},
    'num_images_recent_predicted': {'threshold': [40.0, 'lower']},
    'num_images_total_ground_truth': {'threshold': [5.0, 'lower']},
    'num_images_total_predicted': {'threshold': [40.0, 'lower']}},
   'volume_display_name': 'AdditionalDeepLIIFVolume'},
  'segmentation_metrics': {'dir_gt': 'DeepLIIF_Datasets/model_eval/gt_images',
   'dir_pred': 'DeepLIIF_Datasets/model_eval/model_images',
   'most_recent': 5,
   'thresholds': {'Dice': {'threshold': [20.0, 'lower']},
    'Dice_negative': {'threshold': [0.2, 'lower']},
    'Dice_posi

In [19]:
model_asset_id

## Kick off Deployment & Monitoring Pipeline

In [9]:
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import ws_utils as ws_util

In [ ]:
# pass variables needed in pipeline notebooks through env vars
os.environ['CPD_USERNAME'] = '***'
os.environ['CPD_API_KEY'] = '***'
os.environ['REST_SERVER'] = 'https://wmla-console-cpd-wmla.apps.cpd.mskcc.org/dlim/v1/'
os.environ['DLIM_PATH'] = os.environ['HOME']+'/bin'
os.environ['KERNEL_FILENAME'] = 'kernel-update.py'

os.environ['SERVICE_PROVIDER_NAME'] = SERVICE_PROVIDER_NAME
os.environ['SUBSCRIPTION_NAME'] = subscription_name

os.environ['MODEL_ASSET_ID'] = model_asset_id
os.environ['WML_SPACE_ID'] = WML_SPACE_ID

os.environ['WOS_GUID'] = data_mart_id

In [8]:
paths_nb = ['A2_WMLA_Model_Deploy.ipynb',
            'A3_OpenScale_Configuration.ipynb']

for path_nb in paths_nb:
    print('-'*30,path_nb,'-'*30)
    ws_util.run_pipeline_notebook(path_nb,save_notebook=True)

------------------------------ A2_WMLA_Model_Deploy.ipynb ------------------------------
Executed notebook written to A2_WMLA_Model_Deploy_out.ipynb
Successfully saved data asset content to file: 'tmp/deployment_metadata.yml'

Successfully saved data asset content to file: '/userfs/deploy_submissions/deepliif-base/wmla_edi_utils.py'

Successfully saved data asset content to file: '/userfs/deploy_submissions/deepliif-base/storage_volume_utils.py'

Successfully saved data asset content to file: '/userfs/deploy_submissions/deepliif-base/cpd_utils.py'

Successfully saved data asset content to file: '/userfs/deploy_submissions/deepliif-base/wml_sdk_utils.py'
Found 6 data assets with name deepliif-base.zip, only the first one will be downloaded.

Successfully saved data asset content to file: '/userfs/deploy_submissions/deepliif-base/deepliif-base.zip'

CompletedProcess(args='unzip -j deepliif-base.zip; rm deepliif-base.zip', returncode=0)
Writing /userfs/deploy_submissions/deepliif-base/mod